In [2]:
## Terminal: scrapy crawl ebay_motors2 -o 'cars2.csv'
import csv
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', -1)

# read in data
df = pd.read_csv('cars3.csv')

# filter duplicates
df = df.drop_duplicates()

# fill zeroes for num_bids in non-auctions
df.num_bids.fillna(0,inplace=True)
df.num_bids = pd.to_numeric(df.num_bids,errors='coerce')

df.describe()

,num_bids
count,32518.000000
mean,15.258226
std,18.598035
min,0.000000
25%,0.000000
50%,7.000000
75%,27.000000
max,245.000000


In [34]:
# parse categories/values column into list
df.catsvals = df.catsvals.astype(str)
df.catsvals = [cat.split(',') for cat in df.catsvals]

# remove blank list entries from cat/val column
catsvals = []
for index,row in df.iterrows():
    catval = [x for x in df.catsvals[index] if len(x)>0 and x !='“' and x!='”']
    catsvals.append(catval)
df.catsvals = catsvals

# concat mileage numbers parsed by comma above
catsvals = []
for lst in df.catsvals:
    if 'Mileage:' in lst:
        mileage = lst.index('Mileage:')
        a = lst[:mileage+1] # list before mileage numbers
        
        b = ''
        for n in range(1,4):
            if lst[mileage+n].isdigit():
                b = b+(lst[mileage+n])
            
        b = [lst[mileage+1]+lst[mileage+2]]
        c = lst[mileage+3:] # list after mileage numbers

        abc = a + b + c

        catsvals.append(abc)
    else:
        catsvals.append(lst)
df.catsvals = catsvals
df.shape
df.head()

,title,subtitle,winningbid,buy_it_now_price,num_bids,url,catsvals,cats,vals,catcount,valcount
0,2015 Fiat 500 Pop,2015 fiat 500 pop only 39500 miles automatic rare blue color,NaN,"US $6,900.00",0.0,http://www.ebay.com/itm/2015-Fiat-500-Pop-/282547966266?hash=item41c92bd13a:g:IUoAAOSwbopZT~jY&vxp=mtr,"[['Condition:', 'Used', 'Year:', '2015', 'VIN (Vehicle Identification Number):', '3c3cffar4ft657203', 'Mileage:', '39500', 'Transmission:', 'Automatic', 'Make:', 'Fiat', 'Body Type:', 'Hatchback', 'Model:', '500', 'Warranty:', 'Vehicle does NOT have an existing warranty', 'Trim:', 'Pop', 'Vehicle Title:', 'Clear', 'Engine:', '1.4', 'Options:', 'CD Player', 'Drive Type:', 'FWD', 'Safety Features:', 'Anti-Lock Brakes', 'Power Options:', 'Air Conditioning', 'Sub Model:', '500', 'Fuel Type:', 'Gasoline', 'Exterior Color:', 'Blue', 'For Sale By:', 'Private Seller', 'Interior Color:', 'Black', 'Disability Equipped:', 'No', 'Number of Cylinders:', '4', 'Drive Side:', 'Left-hand drive']]","[Condition:, Year:, VIN (Vehicle Identification Number):, Mileage:, Transmission:, Make:, Body Type:, Model:, Warranty:, Trim:, Vehicle Title:, Engine:, Options:, Drive Type:, Safety Features:, Power Options:, Sub Model:, Fuel Type:, Exterior Color:, For Sale By:, Interior Color:, Disability Equipped:, Number of Cylinders:, Drive Side:]","[Used, 2015, 3c3cffar4ft657203, 39500, Automatic, Fiat, Hatchback, 500, Vehicle does NOT have an existing warranty, Pop, Clear, 1.4, CD Player, FWD, Anti-Lock Brakes, Air Conditioning, 500, Gasoline, Blue, Private Seller, Black, No, 4, Left-hand drive]",24,24
1,2009 Cadillac CTS,2009 Cadillac CTS,"US $5,100.00",NaN,53.0,http://www.ebay.com/itm/2009-Cadillac-CTS-/322575374598?hash=item4b1afd9906:g:UgYAAOSwRXRZWDQb&vxp=mtr,"[['Condition:', 'Used', 'Year:', '2009', 'VIN (Vehicle Identification Number):', '1G6DF577290125887', 'Mileage:', '116000', 'Body Type:', 'Sedan', 'Make:', 'Cadillac', 'Warranty:', 'Vehicle does NOT have an existing warranty', 'Model:', 'CTS', 'Vehicle Title:', 'Clear', 'Engine:', '3.6L V6', 'Options:', 'CD Player', 'Drive Type:', 'RWD', 'Safety Features:', 'Anti-Lock Brakes', 'Power Options:', 'Air Conditioning', 'Exterior Color:', 'Black', 'Fuel Type:', 'Gasoline', 'Interior Color:', 'Black', 'For Sale By:', 'Private Seller', 'Number of Cylinders:', '6', 'Disability Equipped:', 'No', 'Transmission:', 'Automatic', 'Drive Side:', 'Left-hand drive']]","[Condition:, Year:, VIN (Vehicle Identification Number):, Mileage:, Body Type:, Make:, Warranty:, Model:, Vehicle Title:, Engine:, Options:, Drive Type:, Safety Features:, Power Options:, Exterior Color:, Fuel Type:, Interior Color:, For Sale By:, Number of Cylinders:, Disability Equipped:, Transmission:, Drive Side:]","[Used, 2009, 1G6DF577290125887, 116000, Sedan, Cadillac, Vehicle does NOT have an existing warranty, CTS, Clear, 3.6L V6, CD Player, RWD, Anti-Lock Brakes, Air Conditioning, Black, Gasoline, Black, Private Seller, 6, No, Automatic, Left-hand drive]",22,22
2,2008 GMC Savana LS,2008 LS Used 5.3L V8 16V Automatic RWD Minivan/Van,NaN,"US $9,500.00",0.0,http://www.ebay.com/itm/2008-GMC-Savana-LS-/272717880436?hash=item3f7f409c74:g:w~AAAOSwXetZQZ0D&vxp=mtr,"[['Condition:', 'Used', 'Year:', '2008', 'VIN (Vehicle Identification Number):', '1GKFG154181235170', 'Mileage:', '115123', 'Transmission:', 'Automatic', 'Make:', 'GMC', 'Body Type:', 'Minivan/Van', 'Model:', 'Savana', 'Warranty:', 'Vehicle does NOT have an existing warranty', 'Trim:', 'LS', 'Vehicle Title:', 'Clear', 'Engine:', '5.3L V8 16V', 'Fuel Type:', 'Gasoline', 'Drive Type:', 'RWD', 'For Sale By:', 'Dealer', 'Manufacturer Interior Color:', 'Medium Pewter', 'Exterior Color:', 'White', 'Manufacturer Exterior Color:', 'Summit White', 'Interior Color:', 'Gray', 'Number of Doors:', '3 Doors', 'Number of Cylinders:', '8']]","[Condition:, Year:, VIN (Vehicle Identification Number):, Mileage:, Transmission:, Make:, Body Type:, Model:, Warranty:, Trim:, Vehicle Title:, Engine:, Fuel Type:, Dr

In [3]:
# delete values with leading spaces (this is for options fields which need to be lists--put this back in future)
catsvals = []
for lst in df.catsvals:
    clean = [obj for obj in lst if not obj.startswith(' ')]
    catsvals.append(clean)
    
df.catsvals = catsvals
df.shape

(32518, 7)

In [4]:
def catvalsplit(index):
    cats = df.catsvals[index][::2]
    vals = df.catsvals[index][1::2]

    return [cats, vals]

cats = []
vals = []
for index,row in df.iterrows():
    
    cat, val = catvalsplit(index)
    cats.append(cat)
    vals.append(val)
    
df['cats'] = cats
df['vals'] = vals

catcount = []
valcount = []
for index,row in df.iterrows():
    cat = len(df.cats[index])
    val = len(df.vals[index])
    
    catcount.append(cat)
    valcount.append(val)

df['catcount'] = catcount
df['valcount'] = valcount

df.shape

(32518, 11)

In [5]:
# rename dataframe 'cars' and filter out records with different cat/val counts
cars = df[df['catcount']==df['valcount']]
cars = cars.reset_index(drop=True)

# add formatted price and auction (0/1) columns
auction = []
for i in cars.winningbid:
    if isinstance(i,str):
        auction.append(1)
    else:
        auction.append(0)
cols = ['auction']
df1 = pd.DataFrame(auction, columns=cols)
df1 = df1.auction.astype(int)

cars['price'] = cars['winningbid'].fillna(cars['buy_it_now_price'])

cars.reset_index(drop=True, inplace=True)
df1.reset_index(drop=True, inplace=True)

cars = pd.concat([cars, df1], axis=1,ignore_index=False)

cars.price = cars.price.str.strip('US $')
cars.price = cars.price.str.replace(',','')
cars.price = pd.to_numeric(cars.price)

cars = cars[['title','subtitle','url','price','auction','num_bids','cats','vals']]
cars.shape

(29362, 8)

In [6]:
# split categories out into columns and populate with values
def carlookup(index):
    ser1 = pd.Series(cars.cats[index],name='cats')
    ser2 = pd.Series(cars.vals[index],name='vals')

    result = zip(ser1,ser2)
    
    dictionary = {key:value for (key, value) in result}

    return dictionary

dicts = []
for index,row in cars.iterrows():
    dct = carlookup(index)
    dicts.append(dct)

cars['dicts'] = dicts
cars.dicts.apply(pd.Series)

cars = pd.concat([cars.drop(['dicts'], axis=1), cars['dicts'].apply(pd.Series)], axis=1)
cars.shape

(29362, 839)

In [7]:
# Drop low-count columns
cars = cars.dropna(thresh=100,axis=1)
cars.shape

(29362, 45)

In [8]:
cars.columns

Index(['title', 'subtitle', 'url', 'price', 'auction', 'num_bids', 'cats',
       'vals', 'Body Type:', 'CALL NOW:', 'Condition:', 'Disability Equipped:',
       'Doors:', 'Drive Side:', 'Drive Type:', 'Drivetrain:',
       'Engine Description:', 'Engine:', 'Exterior Color:', 'For Sale By:',
       'Fuel Type:', 'Fuel:', 'Inspection:', 'Interior Color:', 'Make:',
       'Manufacturer Exterior Color:', 'Manufacturer Interior Color:',
       'Maximum Seating:', 'Mileage:', 'Model:', 'Number Of Doors:',
       'Number of Cylinders:', 'Number of Doors:', 'Options:',
       'Power Options:', 'Safety Features:', 'Seller Notes:', 'Sub Model:',
       'Transmission:', 'Trim:', 'VIN (Vehicle Identification Number):',
       'VIN:', 'Vehicle Title:', 'Warranty:', 'Year:'],
      dtype='object')

In [9]:
drop = ['cats','vals','Options:','Power Options:','Safety Features:','CALL NOW:']

for col in drop:
    try:
        cars = cars.drop(col,1)
    except ValueError:
        print('Already dropped', col)

In [10]:
# clean up and rename columns
catnames = []
for cat in cars.columns:
    cat = cat.split()
    cat = "".join(cat)
    cat = cat.strip(':')
    cat = cat.replace('/','Or')
    cat = cat.replace(' ','')
    cat = cat.replace('(','_')
    cat = cat.replace(')','')
    cat = cat.strip('?')
    catnames.append(cat)
cars.columns = catnames

In [11]:
#cars.NumberofDoors.value_counts(dropna=False) ### RUN THIS ON ALL COLUMNS

In [12]:
## COLUMNS TO CLEAN & ADD
# displacement in cubic inches (strip and convert liters)

# uniques = cars.Model.astype(str).unique()
# uniques = sorted(uniques,reverse=True)
# uniques

# cars.Model.value_counts(dropna=False)

### Create Dummy Variables

In [13]:
# BodyType categories
cars.BodyType = [x.lower() for x in cars.BodyType.astype(str)]

vans = [x for x in cars.BodyType if 'van' in x]
hatches = [x for x in cars.BodyType if 'hatch' in x]
pickups = [x for x in cars.BodyType if 'pick' in x]
twodoors = [x for x in cars.BodyType if '2 door' in x or '2 dr' in x or '2-door' in x or '2d' in x]
fourdoors = [x for x in cars.BodyType if '4 door' in x or '4 dr' in x or '4-door' in x or '4d' in x or 'four door' in x]
convertibles = [x for x in cars.BodyType if 'convert' in x]
hardtops = [x for x in cars.BodyType if 'hardt' in x or 'hard t' in x]
coupes = [x for x in cars.BodyType if 'coup' in x]
suvs = [x for x in cars.BodyType if 'sport ut' in x or 'sport/ut' in x or 'suv' in x]
ttops = [x for x in cars.BodyType if 't top' in x or 't-top' in x or 'targ' in x]
sedans = [x for x in cars.BodyType if 'sedan' in x]
wagons = [x for x in cars.BodyType if 'wagon' in x]

cars['van'] = [1 if x in vans else 0 for x in cars.BodyType]
cars['hatch'] = [1 if x in hatches else 0 for x in cars.BodyType]
cars['pickup'] = [1 if x in pickups else 0 for x in cars.BodyType]
cars['twodoor'] = [1 if x in twodoors else 0 for x in cars.BodyType]
cars['fourdoor'] = [1 if x in fourdoors else 0 for x in cars.BodyType]
cars['convertible'] = [1 if x in convertibles else 0 for x in cars.BodyType]
cars['hardtop'] = [1 if x in hardtops else 0 for x in cars.BodyType]
cars['coupe'] = [1 if x in coupes else 0 for x in cars.BodyType]
cars['suv'] = [1 if x in suvs else 0 for x in cars.BodyType]
cars['ttop'] = [1 if x in ttops else 0 for x in cars.BodyType]
cars['sedan'] = [1 if x in sedans else 0 for x in cars.BodyType]
cars['wagon'] = [1 if x in wagons else 0 for x in cars.BodyType]

pd.concat([cars.BodyType,cars[cars.columns[-12:]]],axis=1)
cars.shape

(29362, 51)

In [14]:
# NumberofCylinders categories
cars.NumberofCylinders = [x.lower() for x in cars.NumberofCylinders.astype(str)]

cylinders = []
for x in cars.NumberofCylinders:
    if x == '12' or 'v12' in x:
        cyl = 12
    elif x == '10' or 'v10' in x:
        cyl = 10
    elif x == '8' or 'v8' in x or 'v-8' in x or 'eight' in x or '8 cyl' in x:
        cyl = 8
    elif x == '6' or 'v6' in x or 'v-6' in x or 'six' in x or '6 cyl' in x or '6 ' in x:
        cyl = 6
    elif x == '5':
        cyl = 5    
    elif x == '4' or 'v4' in x or 'v-4' in x or 'four' in x or '4 cyl' in x:
        cyl = 4
    elif x == '3' or 'three' in x:
        cyl = 3
    elif x == '2':
        cyl = 2
    else:
        cyl = np.nan
    cylinders.append(cyl)
    
cars['cylinders'] = cylinders
pd.concat([cars.NumberofCylinders,cars[cars.columns[-1:]]],axis=1)
# cars.cylinders.value_counts(dropna=False) 
cars.shape

(29362, 52)

In [15]:
# DriveType categories
cars.DriveType = [x.lower() for x in cars.DriveType.astype(str)]

awd = [x for x in cars.DriveType if 
        'xi' in x
       or 'quattro' in x
       or '4wd' in x
       or 'awd' in x
       or '4x4' in x
       or '4 WD' in x
]
fwd = [x for x in cars.DriveType if 
          'front' in x
       or 'fwd' in x
         ]
rwd = [x for x in cars.DriveType if 
          'rwd' in x
       or 'rear' in x
       or 'rrd' in x
       or  x == 'r'
         ]

cars['awd'] = [1 if x in awd else 0 for x in cars.DriveType]
cars['fwd'] = [1 if x in fwd else 0 for x in cars.DriveType]
cars['rwd'] = [1 if x in rwd else 0 for x in cars.DriveType]

pd.concat([cars.DriveType,cars[cars.columns[-3:]]],axis=1)
cars.shape

(29362, 55)

In [16]:
# ForSaleBy categories
cars.ForSaleBy = [x.lower() for x in cars.ForSaleBy.astype(str)]

private = [x for x in cars.ForSaleBy if 
           'private' in x
           or 'owner' in x
           or 'dwn' in x
           or 'enth' in x
           or 'richard' in x
           or 'burzinski' in x
          and 'dealer' not in x]
dealer = [x for x in cars.ForSaleBy if 
          'dealer' in x
         or 'regist' in x
         or 'licens' in x
          or ' of ' in x
          or 'wholesale' in x
          and 'private' not in x
         ]

cars['private'] = [1 if x in private else 0 for x in cars.ForSaleBy]
cars['dealer'] = [1 if x in dealer else 0 for x in cars.ForSaleBy]

pd.concat([cars.ForSaleBy,cars[cars.columns[-2:]]],axis=1)
cars.shape

(29362, 57)

In [17]:
# Make categories
cars['Make2'] = cars.Make

# Filter out makes with few models???
# cars.Make.value_counts(dropna=False) 

cars = pd.get_dummies(cars, columns=['Make'])
cars = cars.rename(columns = {'Make2':'make'})
# cars = pd.concat([cars, pd.get_dummies(cars, columns=['Make'])], axis=1)

In [18]:
# clean up and rename columns again
catnames = []
for cat in cars.columns:
    cat = cat.split()
    cat = "".join(cat)
    cat = cat.strip(':')
    cat = cat.replace('/','Or')
    cat = cat.strip(' ')
    cat = cat.replace('(','_')
    cat = cat.replace('-','_')
    cat = cat.replace(')','')
    cat = cat.strip('?')
    catnames.append(cat)
cars.columns = catnames

In [19]:
# DriveSide category
cars.DriveSide = [x.lower() for x in cars.DriveSide.astype(str)]

leftdrive = [x for x in cars.DriveSide 
            if 'left' in x 
            or 'lhd' in x 
            or 'usa' in x 
            or 'americ' in x 
            or 'lh' in x 
            or 'l/s' in x 
            and 'right' not in x
            and 'rhd' not in x]
rightdrive = [x for x in cars.DriveSide 
            if 'right' in x 
            or 'rhd' in x 
            or 'uk' in x 
            or 'u.k.' in x 
            or 'australia' in x 
            or 'jdm' in x 
            or 'japan' in x 
            and 'left' not in x
            and 'lhd' not in x
             and 'usa' not in x
             and 'lh' not in x
             and 'l/s' not in x]

cars['leftdrive'] = [1 if x in leftdrive else 0 for x in cars.DriveSide]
cars['rightdrive'] = [1 if x in rightdrive else 0 for x in cars.DriveSide]

pd.concat([cars.DriveSide,cars[cars.columns[-2:]]],axis=1)
cars.shape

(29362, 106)

In [20]:
# Warranty categories
cars.Warranty = [x.lower() for x in cars.Warranty.astype(str)]

warranty = [x for x in cars.Warranty 
            if 'day' in x 
            or 'mile' in x 
            or 'has' in x 
            or 'month' in x 
            or 'year' in x 
            or 'factory' in x 
            and 'expired' not in x
            and 'does not have' not in x
            and 'as is' not in x]

cars['warranty'] = [1 if x in warranty else 0 for x in cars.Warranty]

pd.concat([cars.Warranty,cars[cars.columns[-1:]]],axis=1)
cars.shape

(29362, 107)

In [21]:
# Transmission categories
cars.Transmission = [x.lower() for x in cars.Transmission.astype(str)]

manual = [x for x in cars.Transmission if 'manual' in x or 'stick' in x]
auto = [x for x in cars.Transmission if 'auto' in x or 'pdk' in x or 'DCT' in x or 'matic' in x or 'cvt' in x]

cars['manual'] = [1 if x in manual else 0 for x in cars.Transmission]
cars['auto'] = [1 if x in auto else 0 for x in cars.Transmission]

pd.concat([cars.Transmission,cars[cars.columns[-2:]]],axis=1)
cars.shape

(29362, 109)

In [22]:
# Condition categories
cars.Condition = [x.lower() for x in cars.Condition.astype(str)]

used = [x for x in cars.Condition if 'used' in x]
new = [x for x in cars.Condition if 'new' in x]
cpo = [x for x in cars.Condition if 'pre-owned' in x or 'preowned' in x]

cars['used'] = [1 if x in used else 0 for x in cars.Condition]
cars['new'] = [1 if x in new else 0 for x in cars.Condition]
cars['cpo'] = [1 if x in cpo else 0 for x in cars.Condition]

pd.concat([cars.Condition,cars[cars.columns[-3:]]],axis=1)
cars.shape

(29362, 112)

In [23]:
cars.VehicleTitle.unique()

array(['Clear', 'Salvage', 'Rebuilt', 'Lemon & Manufacturer Buyback', nan,
       'Flood', 'salvage', 'N/A'], dtype=object)

In [24]:
# VehicleTitle categories
cars.VehicleTitle = [x.lower() for x in cars.VehicleTitle.astype(str)]

title_clear = [x for x in cars.VehicleTitle if 'clear' in x]
title_salvage = [x for x in cars.VehicleTitle if 'salvage' in x]
title_rebuilt = [x for x in cars.VehicleTitle if 'rebuilt' in x]
title_flood = [x for x in cars.VehicleTitle if 'flood' in x]
title_lemon = [x for x in cars.VehicleTitle if 'lemon' in x]

title_notclear = [x for x in cars.VehicleTitle if 'salvage' in x or 'rebuilt' in x or 'flood' in x or 'lemon' in x]

cars['title_clear'] = [1 if x in title_clear else 0 for x in cars.VehicleTitle]
cars['title_salvage'] = [1 if x in title_salvage else 0 for x in cars.VehicleTitle]
cars['title_rebuilt'] = [1 if x in title_rebuilt else 0 for x in cars.VehicleTitle]
cars['title_flood'] = [1 if x in title_flood else 0 for x in cars.VehicleTitle]
cars['title_lemon'] = [1 if x in title_lemon else 0 for x in cars.VehicleTitle]

# Create category for non-clear vehicle titles

cars['title_is_clear'] = [1 if x not in title_notclear else 0 for x in cars.VehicleTitle]

pd.concat([cars.VehicleTitle,cars[cars.columns[-5:]]],axis=1)
cars.shape

(29362, 118)

In [25]:
# Years as age
cars.Year = pd.to_numeric(cars.Year, errors='coerce')
cars['age'] = [2017 - x for x in cars.Year]
cars['age']= cars.age.mask(cars.age == 0,0.0001)
cars = cars.rename(columns = {'Year':'year'})
cars.shape

(29362, 119)

In [26]:
## Pull years from title where missing from year column
# fix_years = cars[['title','year']]
# fix_years[fix_years.isnull().any(axis=1)]

In [27]:
# Mileage as float
cars.Mileage = pd.to_numeric(cars.Mileage, errors='coerce')
cars = cars.rename(columns = {'Mileage':'mileage'})
cars.shape

(29362, 119)

In [28]:
# FuelType categories
cars.FuelType = [x.lower() for x in cars.FuelType.astype(str)]

gas = [x for x in cars.FuelType if 'gas' in x or 'flex' in x or 'unlead' in x or 'prem' in x or 'regular' in x or 'octane' in x]
diesel = [x for x in cars.FuelType if 'dies' in x or 'deis' in x]
electric = [x for x in cars.FuelType if 'elec' in x]
ethanol = [x for x in cars.FuelType if 'eth' in x or 'e85' in x]
cng = [x for x in cars.FuelType if 'ng' in x]

cars['gas'] = [1 if x in gas else 0 for x in cars.FuelType]
cars['diesel'] = [1 if x in diesel else 0 for x in cars.FuelType]
cars['electric'] = [1 if x in electric else 0 for x in cars.FuelType]
cars['ethanol'] = [1 if x in ethanol else 0 for x in cars.FuelType]
cars['cng'] = [1 if x in cng else 0 for x in cars.FuelType]

pd.concat([cars.FuelType,cars[cars.columns[-5:]]],axis=1)
cars.shape

(29362, 124)

In [29]:
# Add counts of category columns to filter out non-responsive records
bodytype_list = ['van','hatch','pickup','twodoor','fourdoor','convertible','hardtop','coupe','suv','ttop','sedan','wagon']
drivetype_list = ['awd','fwd','rwd']
forsaleby_list = ['private','dealer']
driveside_list = ['leftdrive','rightdrive']
transmission_list = ['manual','auto']
condition_list = ['used','new','cpo']
vehicletitle_list = ['title_clear','title_salvage','title_rebuilt','title_flood','title_lemon']
fuel_list = ['gas','diesel','electric','ethanol','cng']
# cylinders -- filter out NaN
# year -- filter out NaN
# mileage -- filter out NaN

cars['bodytype_ct'] = cars[bodytype_list].sum(axis=1)
cars['drivetype_ct'] = cars[drivetype_list].sum(axis=1)
cars['forsaleby_ct'] = cars[forsaleby_list].sum(axis=1)
cars['driveside_ct'] = cars[driveside_list].sum(axis=1)
cars['transmission_ct'] = cars[transmission_list].sum(axis=1)
cars['condition_ct'] = cars[condition_list].sum(axis=1)
cars['vehicletitle_ct'] = cars[vehicletitle_list].sum(axis=1)
cars['fuel_ct'] = cars[fuel_list].sum(axis=1)

cars.shape

(29362, 132)

In [30]:
# Drop more columns # Make & Model excluded for now
drop = ['BodyType','Condition','DisabilityEquipped','Doors','DriveSide','DriveType', 
'Drivetrain', 'EngineDescription','ExteriorColor', 'ForSaleBy', 'FuelType', 'Fuel',
'Engine','InteriorColor','ManufacturerExteriorColor','Inspection', 'MaximumSeating',
'ManufacturerInteriorColor', 'NumberofCylinders',
'NumberofDoors', 'SellerNotes', 'SubModel', 'Transmission','Trim',
'VIN_VehicleIdentificationNumber', 'VIN', 'VehicleTitle', 'Warranty','NumberOfDoors']

for col in drop:
    try:
        cars = cars.drop(col,1)
    except ValueError:
        print('Already dropped', col)
        
cars = cars.rename(columns = {'Model':'model'})

In [31]:
cars.to_pickle('cars_clean.pkl')

In [32]:
cars.columns[20:]

Index(['wagon', 'cylinders', 'awd', 'fwd', 'rwd', 'private', 'dealer', 'make',
       'Make_Acura', 'Make_AlfaRomeo', 'Make_AstonMartin', 'Make_Audi',
       'Make_BMW', 'Make_Bentley', 'Make_Buick', 'Make_Cadillac',
       'Make_Chevrolet', 'Make_Chrysler', 'Make_Dodge', 'Make_Fiat',
       'Make_Ford', 'Make_GMC', 'Make_Honda', 'Make_Hummer', 'Make_Hyundai',
       'Make_Infiniti', 'Make_Jaguar', 'Make_Jeep', 'Make_Kia',
       'Make_LandRover', 'Make_Lexus', 'Make_Lincoln', 'Make_Lotus',
       'Make_Maserati', 'Make_Mazda', 'Make_Mercedes_Benz', 'Make_Mercury',
       'Make_Mini', 'Make_Mitsubishi', 'Make_Nissan', 'Make_Oldsmobile',
       'Make_Plymouth', 'Make_Pontiac', 'Make_Porsche', 'Make_Ram',
       'Make_Rolls_Royce', 'Make_Saab', 'Make_Saturn', 'Make_Scion',
       'Make_Smart', 'Make_Subaru', 'Make_Tesla', 'Make_Toyota',
       'Make_Volkswagen', 'Make_Volvo', 'leftdrive', 'rightdrive', 'warranty',
       'manual', 'auto', 'used', 'new', 'cpo', 'title_clear', 'title_salva

In [33]:
cars.NumberOfDoors.value_counts(dropna=False) ### RUN THIS ON ALL COLUMNS

AttributeError: 'DataFrame' object has no attribute 'NumberOfDoors'